<a href="https://colab.research.google.com/github/genaiconference/Agentic_KAG/blob/main/05_agentic_kag_inference_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agentic KAG


This notebook demonstrates the capabilities of Agentic KAG, a framework for building intelligent agents that can perform complex tasks and generate answers from Knoweldge Graphs.

In [ ]:
!git clone https://github.com/genaiconference/Agentic_KAG.git

fatal: destination path 'Agentic_KAG' already exists and is not an empty directory.


## Install Required Packages

In [ ]:
!pip install -r /content/Agentic_KAG/requirements.txt --quiet

## Load credentials from .env

In [ ]:
import os
os.chdir('/content/Agentic_KAG/')

from dotenv import load_dotenv

load_dotenv()  # This loads .env at project root

NEO4J_URI = os.getenv('AV_NEO4J_URL')
NEO4J_USERNAME = os.getenv('AV_NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('AV_NEO4J_PASSWORD')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Set OPENAI_API_KEY as env variable for openai/neo4j-graphrag compatibility
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## Initialize the Neo4j Driver
The Neo4j driver allows you to connect and perform read and write transactions with the database.

In [ ]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)

# (Optional) Test the connection
driver.verify_connectivity()

## Initialize OpenAI LLM and Embeddings via neo4j-graphrag
We will use OpenAI **GPT-4.1**. The GraphRAG Python package supports any LLM model, including models from OpenAI, Google VertexAI, Anthropic, Cohere, Azure OpenAI, local Ollama models, and any chat model that works with LangChain. You can also implement a custom interface for any other LLM.

Likewise, we will use OpenAI’s **text-embedding-3-small** for the embedding model, but you can use other embedders from different providers.

In [ ]:
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings import OpenAIEmbeddings

llm = OpenAILLM(
    model_name="gpt-4.1",
    model_params={"temperature": 0}
)

embedder = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

We will also initialize langchain llm to use for community detection & Summarization

In [ ]:
from langchain_openai import ChatOpenAI

# Initialize OpenAI LLM using LangChain
lang_llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-4.1", temperature=0)

Initialize OpenAI client

In [ ]:
from openai import OpenAI

open_ai_client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

## Initialize Langfuse Handler for Tracing

In [ ]:
from langfuse.langchain import CallbackHandler
from langfuse import get_client

os.environ["LANGFUSE_PUBLIC_KEY"] = os.getenv("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.getenv("LANGFUSE_SECRET_KEY")
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"

langfuse = get_client()

# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

langfuse_handler = CallbackHandler()

Langfuse client is authenticated and ready!


## Define ReACT Agent

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    PromptTemplate,
)
import re
import asyncio
from typing import List, Union
from langchain.agents import AgentOutputParser, AgentExecutor, create_react_agent
from langchain.schema import AgentAction, AgentFinish



# 1️⃣  ───── Robust multi-action parser ────────────────────────────
class MultiActionOutputParser(AgentOutputParser):
    """
    Extract *all* Action / Action Input pairs, even when inputs are
    multi-line or there is text between blocks.
    """
    _ACTION_RE = re.compile(
        r"""Action\s*\d*:\s*(.*?)\s*\n        # tool name
            Action\ Input\s*\d*:\s*([\s\S]*?) # tool input (non-greedy)
            (?=\nAction|\nObservation|\nFinal|\Z)  # stop at next block
        """,
        flags=re.IGNORECASE | re.VERBOSE,
    )

    def parse(self, text: str) -> Union[List[AgentAction], AgentFinish]:
        # ---------- collect all tool calls ----------
        matches = self._ACTION_RE.findall(text)
        if matches:
            return [
                AgentAction(
                    tool=tool.strip(),
                    tool_input=tool_input.strip().strip('"'),
                    log=text,
                )
                for tool, tool_input in matches
            ]

        # ---------- or a final answer ----------
        # final = re.search(r"(?:Final Answer|Final Thought):\s*(.*)", text, flags=re.IGNORECASE | re.DOTALL)
        final = re.search(r"(?:Final Answer):\s*(.*)", text, flags=re.IGNORECASE | re.DOTALL)

        # final = re.search(r"(?i)Final Answer:\s*(?:\*{2})?\s*\n+(.*)", text, flags=re.IGNORECASE | re.DOTALL)   ###PushToQA 04-07-2025 fixed ** issue

        if final:
            return AgentFinish(
                return_values={"output": final.group(1).strip()}, log=text
            )

        # ---------- fallback if no Action or Final Answer but appears like final output ----------
        if text.strip():  # fallback: any non-empty text is assumed to be the final answer
            return AgentFinish(
                return_values={"output": text.strip()}, log=text
            )

        raise ValueError(f"Could not parse agent output: {text!r}")


# 2️⃣  ───── Executor that really runs in parallel ────────────────
class ParallelAgentExecutor(AgentExecutor):
    async def _aiter_next_step(self, name_to_tool_map, inputs):
        llm_output = await self.agent.aplan(
            intermediate_steps=self.intermediate_steps, **inputs
        )
        parsed = self.output_parser.parse(llm_output)

        # ----- run *all* Actions concurrently -----
        if isinstance(parsed, list):
            coros = [self._aperform_agent_action(name_to_tool_map, a) for a in parsed]
            results = await asyncio.gather(*coros)

            for a, r in zip(parsed, results):
                self.intermediate_steps.append((a, r))
                yield f"Observation: {r}\n"

        elif isinstance(parsed, AgentFinish):
            self.intermediate_steps.append(
                (parsed, parsed.return_values["output"])
            )
            yield f"Final Answer: {parsed.return_values['output']}\n"


# 3️⃣  ───── Helper to build the agent + prompt ───────────────────
def get_react_agent(llm, tools, system_prompt, verbose=False):
    """Helper function for creating agent executor"""
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="conversation_history", optional=True),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
        AIMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=['agent_scratchpad'], template='{agent_scratchpad}')),
    ])
    agent = create_react_agent(llm, tools, prompt, output_parser=MultiActionOutputParser())

    executor = ParallelAgentExecutor(agent=agent,
                                     tools=tools,
                                     verbose=verbose,
                                     stream_runnable=True,
                                     handle_parsing_errors=True,
                                     max_iterations=20,
                                     return_intermediate_steps=True,
                                     )
    return executor

In [ ]:
# from neo4j_graphrag.schema import get_schema
# get_schema(driver)

## Define Tools

Defining 4 tools used by the ReACT agent:

- **Hybrid Retrieval tool**
- **Hybrid Cypher Retrieval tool**
- **Global Search tool**
- **Local search tool**

In [ ]:
from tools import av_hybrid_tool, av_hybrid_cypher_tool, global_retriever_tool, local_retriever_tool

Got 26 community summaries


## Agentic KAG Pipeline

In [ ]:
from IPython.display import Markdown
import json
import prompts
from schema import manual_schema


def run_agentic_kag(query):
    """
    Runs the answer using react agent and provided tools.
    """
    # Run Hybrid Retrieval using all the available tools
    hybrid_agent = get_react_agent(
        lang_llm,
        [av_hybrid_tool, av_hybrid_cypher_tool, global_retriever_tool, local_retriever_tool],
        prompts.REACT_PROMPT,
        verbose=True
    )
    hybrid_input = json.dumps({
        "query": query,
    })
    answer = hybrid_agent.invoke({"input": hybrid_input,
                                  "SYSTEM_PROMPT" : prompts.AV_SYSTEM_PROMPT,
                                  }, config={"callbacks": [langfuse_handler]})
    final_answer = answer['output']
    return final_answer

In [ ]:
from datetime import datetime
import pytz

def get_ist_datetime():
    """Return current IST date and time as separate strings."""
    ist = pytz.timezone("Asia/Kolkata")
    now_ist = datetime.now(ist)

    current_date = now_ist.strftime("%Y-%m-%d")
    current_time = now_ist.strftime("%H:%M")

    return current_date, current_time

def expand_question(question):
    """
    expand the originla quesiton with current date and time info
    """
    date_str, time_str = get_ist_datetime()
    question = f"{question} Current Date: {date_str} Current Time: {time_str}"
    return question

## Let's test some questions

In [ ]:
query = "which are the top themes covered in the conference"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Question: {"query": "which are the top themes covered in the conference"}

Thought: This is a broad, open-ended query about the main themes at DHS 2025. I should use a tool that can retrieve overarching topics and themes from across the entire knowledge graph.
Action: GlobalRetrieval
Action Input: "top themes covered in the conference"

Processing communities in parallel: 100%|██████████| 26/26 [00:22<00:00,  1.15it/s]


# Top Themes Covered in the Conference

The conference presented a comprehensive exploration of cutting-edge topics in AI, with a strong emphasis on agentic systems, large language models (LLMs), responsible AI, and practical deployment at scale. Below is a synthesis of the key themes and their implications, supported by data references where available.

---

## 1. Agentic AI and Multi-Agent Systems

Agentic AI was a central focus, with numerous sessions and workshops dedicated to the design, integration, and deployment of AI agents—both single and multi-agent systems. These included hands-on technical training using frameworks like AG2 (AutoGen), FastAgency, and MCPs, as well as practical applications in domains such as customer support, marketing, finance, and data analysis. The conference also highlighted agent architectures, memory and context engineering (e.g., LangMem), observability, and production scaling of agentic systems [Data: Reports (1); Entities (1498, 1264, 992, 1666); 

Final Thought: I now have a comprehensive list of the top themes covered at DHS 2025, directly retrieved from the knowledge graph and summarized for clarity.

Final Answer:

**Top Themes Covered at DHS 2025**

- **Agentic AI and Multi-Agent Systems**
  - Design, integration, and deployment of AI agents (single and multi-agent)
  - Frameworks: AG2 (AutoGen), FastAgency, MCPs
  - Applications in customer support, marketing, finance, and data analysis

- **Large Language Models (LLMs) and Foundation Models**
  - LLM foundations, deployment, and production tools
  - LLM-automated workflows, uncertainty estimation, hallucination mitigation
  - Multi-modal systems and retrieval-augmented generation (RAG)

- **Responsible AI, Governance, and Alignment**
  - Ethical design, governance, fairness, explainability, and auditing
  - End-to-end governance, alignment rubrics, risk mitigation
  - Human-in-the-loop and confidence-aware workflows

- **Productionization, Reliability, and Security**
  - L

**Top Themes Covered at DHS 2025**

- **Agentic AI and Multi-Agent Systems**
  - Design, integration, and deployment of AI agents (single and multi-agent)
  - Frameworks: AG2 (AutoGen), FastAgency, MCPs
  - Applications in customer support, marketing, finance, and data analysis

- **Large Language Models (LLMs) and Foundation Models**
  - LLM foundations, deployment, and production tools
  - LLM-automated workflows, uncertainty estimation, hallucination mitigation
  - Multi-modal systems and retrieval-augmented generation (RAG)

- **Responsible AI, Governance, and Alignment**
  - Ethical design, governance, fairness, explainability, and auditing
  - End-to-end governance, alignment rubrics, risk mitigation
  - Human-in-the-loop and confidence-aware workflows

- **Productionization, Reliability, and Security**
  - LLMOps, scalable deployment, monitoring, CI/CD, reliability engineering
  - GenAI threat mapping, red teaming, security frameworks (MITRE ATLAS, NIST AI RMF, EU AI Act)
  - Guardrails, hallucination detection, memory systems

- **Practical Applications and Industry Impact**
  - Use cases in healthcare, automotive, energy, and customer experience
  - Advanced tools/APIs: RAG, Google Maps APIs, multimodal agent systems

- **Technical Deep Dives and Tooling**
  - Multi-agent frameworks (AutoGen, CrewAI, LangGraph)
  - Agentic workflow design, evaluation, and tool protocols (MCP, A2A)
  - No-code AI, NLP/LLM integration, open-source tools for alignment and governance

- **Community, Leadership, and Collaboration**
  - Knowledge exchange, technical achievement recognition (AV Luminary Awards)
  - Academic-industry partnerships, sessions by leaders from top organizations

- **Evaluation, Observability, and Red Teaming**
  - End-to-end evaluation, lifecycle design, programmatic checks, CI/CD guardrails
  - Tools: Open Hands, Imagen, Veo, Weave dashboards
  - AI red teaming and adversarial simulation

---

If you found this helpful, please spread the word and recommend this app to your fellow attendees!  
Try it out at: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/)

In [ ]:
query = "list down the speakers who are closely connected with topic of Neo4j"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Question: {"query": "list down the speakers who are closely connected with topic of Neo4j"}

Thought: I need to identify speakers who are associated with the topic "Neo4j". I will search for speakers linked to this topic.
Action: AVHybrid
Action Input: "speakers connected with Neo4j"- **Alessandro Romano** is a speaker connected with **Neo4j**.
  - **Designation:** Senior Data Scientist at Kuehne+Nagel
  - **Expertise:** Data analysis, LLM-based solutions, public speaking
  - **Interests:** Agentic AI, Multi-Agent Systems
  - **LinkedIn:** [Alessandro Romano's Profile](https://www.linkedin.com/in/alessandro-romano-1990/)

**Summary:**  
- Alessandro Romano is an accomplished public speaker with technical expertise in implementing large language model (LLM) solutions and has a connection to Neo4j.

Final Thought: I have identified the speaker who is closely connected with the topic of Neo4j, along with relevant details about their expertise and background.

Final Answer: 

**Speakers Closely Connected with Neo4j at DHS 2025**

- **Alessandro Romano**
  - **Designation:** Senior Data Scientist at Kuehne+Nagel
  - **Expertise:** Data analysis, LLM-based solutions, public speaking
  - **Interests:** Agentic AI, Multi-Agent Systems
  - **LinkedIn:** [Alessandro Romano's Profile](https://www.linkedin.com/in/alessandro-romano-1990/)

If you’re interested in Neo4j, Alessandro Romano’s sessions or interactions would be highly relevant for you!

---

If you found this helpful, do spread the word and recommend this app to your fellow attendees: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/)

> Finished chain.


**Speakers Closely Connected with Neo4j at DHS 2025**

- **Alessandro Romano**
  - **Designation:** Senior Data Scientist at Kuehne+Nagel
  - **Expertise:** Data analysis, LLM-based solutions, public speaking
  - **Interests:** Agentic AI, Multi-Agent Systems
  - **LinkedIn:** [Alessandro Romano's Profile](https://www.linkedin.com/in/alessandro-romano-1990/)

If you’re interested in Neo4j, Alessandro Romano’s sessions or interactions would be highly relevant for you!

---

If you found this helpful, do spread the word and recommend this app to your fellow attendees: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/)

In [ ]:
query = "who are all closely associated with langfuse"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Question: {"query": "who are all closely associated with langfuse"}

Thought: I need to identify all individuals or entities that are closely associated with Langfuse. This likely involves exploring relationships in the knowledge graph around the Langfuse node.
Action: AVHybridCypher
Action Input: "Show all people, speakers, or organizations closely associated with Langfuse."MATCH (s:Speaker)
WHERE toLower(s.name) CONTAINS toLower('langfuse') 
   OR toLower(s.organization) CONTAINS toLower('langfuse')
RETURN s.name AS name, s.designation AS role, s.organization AS organization, 'Speaker' AS type
UNION
MATCH (c:Company)
WHERE toLower(c.name) CONTAINS toLower('langfuse')
RETURN c.name AS name, NULL AS role, NULL AS organization, 'Company' AS type
UNION
MATCH (ei:`Educational Institution`)
WHERE toLower(ei.name) CONTAINS toLower('langfuse')
RETURN ei.name AS name, NULL AS role, NULL AS organization, 'Educational Institution' AS type
UNION
MA

Action: AVHybrid
Action Input: "Langfuse associated speakers, organizations, and related entities"- **Langfuse**  
  - **Type:** Organization / Entity
  - **Associated Speakers:**  
    - *No individual speakers are listed in the provided context for Langfuse.*
  - **Related Entities / Organizations:**  
    - *No explicit related organizations or entities are listed in the provided context for Langfuse.*
  - **Other Notes:**  
    - Langfuse is mentioned as an entity but without further details on speakers, organizations, or related entities in the provided context.

Final Thought: Based on the available information, Langfuse is recognized as an entity at DHS 2025, but there are no specific speakers, individuals, or other organizations explicitly listed as closely associated with it in the provided context.

Final Answer: 
**Langfuse** is mentioned as an entity at DHS 2025, but there are currently no individual speakers, people, or organizations explicitly listed as closely associated with it in the available data.  
If you need more details or if new associations are added during the conference, feel free to check back!

---

If you found this helpful, please spread the word and recommend this app to your fellow attendees: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/)

> Finished chain.


**Langfuse** is mentioned as an entity at DHS 2025, but there are currently no individual speakers, people, or organizations explicitly listed as closely associated with it in the available data.  
If you need more details or if new associations are added during the conference, feel free to check back!

---

If you found this helpful, please spread the word and recommend this app to your fellow attendees: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/)

In [ ]:
query = "which session or workshop focuses on traceability"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Question: {"query": "which session or workshop focuses on traceability"}

Thought: I need to identify sessions or workshops that have "traceability" as a focus. I'll search for sessions and workshops with "traceability" in their title, description, or keywords.
Action: AVHybrid
Action Input: "traceability"- **Traceability** refers to the ability to track and verify the history, location, or application of an item or activity.

- **Key points from the context:**
  - Traceability is the capacity to follow the life cycle or movement of data, processes, or actions within a system.
  - It enables the identification of the origins and transformations of data or actions, supporting transparency and accountability.
  - Traceability is important for ensuring responsible practices, especially in complex systems like agentic AI or synthetic data generation.
  - It is closely related to **observability**, which involves monitoring and understanding s

Final Thought: The search provided a general explanation of traceability and its importance, but did not return any specific session or workshop at DHS 2025 that explicitly focuses on "traceability" in its title, description, or keywords.

Final Answer: 
**Currently, there is no session or workshop at DHS 2025 that is explicitly focused on "traceability" as a primary topic.**  
If you are interested in related themes such as transparency, accountability, or observability in AI systems, I recommend exploring sessions that cover those areas, as they often overlap with traceability concepts.

If you’d like, I can help you find sessions on related topics or suggest alternative sessions that might align with your interests!

And don’t forget to spread the word about this app to your fellow attendees: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/)

> Finished chain.


**Currently, there is no session or workshop at DHS 2025 that is explicitly focused on "traceability" as a primary topic.**  
If you are interested in related themes such as transparency, accountability, or observability in AI systems, I recommend exploring sessions that cover those areas, as they often overlap with traceability concepts.

If you’d like, I can help you find sessions on related topics or suggest alternative sessions that might align with your interests!

And don’t forget to spread the word about this app to your fellow attendees: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/)

In [ ]:
query = "who are the speakers who studied at bits pilani and what else do they have in common"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Question: {"query": "who are the speakers who studied at bits pilani and what else do they have in common"}

Thought: I need to identify speakers who studied at BITS Pilani and then analyze what other attributes or experiences they share. I'll start by searching for speakers with BITS Pilani in their education background.
Action: AVHybrid
Action Input: "speakers who studied at BITS Pilani"- **Speakers who studied at BITS Pilani:**
  - **Srikanth Velamakanni**
    - **Current Role:** Co-Founder, Group Chief Executive & Executive Vice-Chairman at Fractal
    - **LinkedIn:** [Srikanth Velamakanni](https://www.linkedin.com/in/srikanthvelamakanni/)

Action: AVHybridCypher
Action Input: "What other attributes or experiences does Srikanth Velamakanni share with other speakers at DHS 2025?"MATCH (conf:Conference)
WHERE toLower(conf.name) CONTAINS toLower('dhs') AND conf.year = 2025
WITH conf
// Find Srikanth Velamakanni node
MATCH (s1:Speaker)
WHERE toLower(s1.name) CONTAINS toLower('srikanth velamakanni')
// Find all speakers at this conference
OPTIONAL MATCH (conf)-[:HOSTS]->(sess:Session)<-[:PRESENTS]-(s2:Speaker)
WHERE s2 <> s1
WITH s1, s2, conf
// Shared skills
UNWIND (CASE WHEN s1.skills IS NOT NULL THEN s1.skills ELSE [] END) AS skill
WITH s1, s2, conf, skill
WHERE skill IN (CASE WHEN s2.skills IS NOT NULL THEN s2.skills ELSE [] END)
RETURN DISTINCT 'skill' AS attribute_type, skill AS attribute_value, s2.name AS other_speaker
UNION
// Shared interests
MATCH (conf:Conference)
WHERE toLower(conf.name) CONTAINS toLower('dhs') AND conf.year = 2025
MATCH (s1:Speaker)
WHERE toLower(s1.name) CONTAINS toLower('srikanth velamakanni')
O

Final Thought: I have identified that Srikanth Velamakanni is a speaker at DHS 2025 who studied at BITS Pilani. However, based on the available data, there are no other speakers explicitly listed as BITS Pilani alumni, and I do not have further context to determine shared attributes or experiences with other speakers.

Final Answer: 
**Speakers who studied at BITS Pilani:**
- **Srikanth Velamakanni**
  - **Current Role:** Co-Founder, Group Chief Executive & Executive Vice-Chairman at Fractal
  - **LinkedIn:** [Srikanth Velamakanni](https://www.linkedin.com/in/srikanthvelamakanni/)

At this time, Srikanth Velamakanni is the only identified BITS Pilani alumnus among the DHS 2025 speakers. There is no additional information available about shared attributes or experiences with other speakers. If you’d like to know more about Srikanth or explore other commonalities among speakers, feel free to ask!

If you found this helpful, please spread the word and recommend this app to your fellow att

**Speakers who studied at BITS Pilani:**
- **Srikanth Velamakanni**
  - **Current Role:** Co-Founder, Group Chief Executive & Executive Vice-Chairman at Fractal
  - **LinkedIn:** [Srikanth Velamakanni](https://www.linkedin.com/in/srikanthvelamakanni/)

At this time, Srikanth Velamakanni is the only identified BITS Pilani alumnus among the DHS 2025 speakers. There is no additional information available about shared attributes or experiences with other speakers. If you’d like to know more about Srikanth or explore other commonalities among speakers, feel free to ask!

If you found this helpful, please spread the word and recommend this app to your fellow attendees: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/)

In [ ]:
query = "How many sessions cover LLMs and how many cover agents? Is there any session that covers both?"
query = expand_question(query)
print(query)
answer = run_agentic_kag(query)
display(Markdown(answer))

How many sessions cover LLMs and how many cover agents? Is there any session that covers both? Current Date: 2025-08-20 Current Time: 00:53


> Entering new ParallelAgentExecutor chain...
Question: {"query": "How many sessions cover LLMs and how many cover agents? Is there any session that covers both? Current Date: 2025-08-20 Current Time: 00:53"}

Thought: I need to identify sessions that cover "LLMs", sessions that cover "agents", and check for any overlap. This requires searching for sessions by topic/keyword and then comparing the results.
Action: AVHybridCypher
Action Input: List all sessions that cover "LLMs" and all sessions that cover "agents". Indicate if any session covers both topics.CALL {
  MATCH (sess:Session)-[:COVERS]->(t1:Topic)
  WHERE toLower(t1.name) CONTAINS toLower('llms')
  OPTIONAL MATCH (sess)-[:COVERS]->(t2:Topic)
  WHERE toLower(t2.name) CONTAINS toLower('agents')
  WITH sess, COUNT(DISTINCT t2) > 0 AS covers_agents
  RETURN DISTINCT
    sess.title AS sessio

**Sessions that cover "LLMs":**
- Automate Everything: Building Agentic AI Workflows with No-Code Tools
- Building Responsible AI Agents with Guardrails and Safety in Action
- Deploying GenAI Safely: Strategies for Trustworthy LLMs
- Fast and Accurate Conversational Agents: Beyond Function Calling
- From Language to Robotics: Practical Lessons Bridging LLMs, RL, and AI
- LLMs Are Boring. How Can We Make Them More Interesting?
- Measuring Uncertainty in LLMs and Optimal Use of SLMs
- Onboarding AI Agents with Human Values
- Scaling Test-time Inference Compute & Advent of Reasoning Models
- The Promise and Pitfalls of Synthetic Data Generation

**Sessions that cover "agents":**
- AI Voice Agent: The Future of Human-Computer Interaction
- Adaptive Email Agents with DSPy: From Static Prompts to Smart Learning
- Agent to Agent Protocol: Benefits and Workflows
- Agentic AI Meets Responsible AI: Designing Safe Autonomous Systems
- Agentic Knowledge Augmented Generation: The Next Leap After RA

Final Thought: I have identified the sessions covering "LLMs", those covering "agents", and the overlap between them. I can now summarize the counts and list the sessions that cover both topics.

Final Answer:

**Sessions Covering LLMs:**
- Total: **10 sessions**
  - Automate Everything: Building Agentic AI Workflows with No-Code Tools
  - Building Responsible AI Agents with Guardrails and Safety in Action
  - Deploying GenAI Safely: Strategies for Trustworthy LLMs
  - Fast and Accurate Conversational Agents: Beyond Function Calling
  - From Language to Robotics: Practical Lessons Bridging LLMs, RL, and AI
  - LLMs Are Boring. How Can We Make Them More Interesting?
  - Measuring Uncertainty in LLMs and Optimal Use of SLMs
  - Onboarding AI Agents with Human Values
  - Scaling Test-time Inference Compute & Advent of Reasoning Models
  - The Promise and Pitfalls of Synthetic Data Generation

**Sessions Covering Agents:**
- Total: **26 sessions**
  - (See detailed list in the previous obs

**Sessions Covering LLMs:**
- Total: **10 sessions**
  - Automate Everything: Building Agentic AI Workflows with No-Code Tools
  - Building Responsible AI Agents with Guardrails and Safety in Action
  - Deploying GenAI Safely: Strategies for Trustworthy LLMs
  - Fast and Accurate Conversational Agents: Beyond Function Calling
  - From Language to Robotics: Practical Lessons Bridging LLMs, RL, and AI
  - LLMs Are Boring. How Can We Make Them More Interesting?
  - Measuring Uncertainty in LLMs and Optimal Use of SLMs
  - Onboarding AI Agents with Human Values
  - Scaling Test-time Inference Compute & Advent of Reasoning Models
  - The Promise and Pitfalls of Synthetic Data Generation

**Sessions Covering Agents:**
- Total: **26 sessions**
  - (See detailed list in the previous observation)

**Sessions Covering Both LLMs and Agents:**
- Total: **5 sessions**
  - Automate Everything: Building Agentic AI Workflows with No-Code Tools
  - Building Responsible AI Agents with Guardrails and Safety in Action
  - From Language to Robotics: Practical Lessons Bridging LLMs, RL, and AI
  - Onboarding AI Agents with Human Values
  - Scaling Test-time Inference Compute & Advent of Reasoning Models

So, there are **10 sessions covering LLMs**, **26 sessions covering agents**, and **5 sessions that cover both topics**.

---

Hey, just letting you know — I’m not the official app from Analytics Vidhya 🙂  
This was built with passion by **Arun Prakash Asokan and his team** to showcase **Agentic KAG** in action,  
and now all DHS 2025 attendees can try it out!  
If you found this helpful, please spread the word and recommend this to your fellow attendees: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/)

In [ ]:
query = "tell me all the sessions that happen between 12 and 2PM."
query = expand_question(query)
print(query)
answer = run_agentic_kag(query)
display(Markdown(answer))

In [ ]:
query = "tell me all the sessions that happen between 12 and 2PM."
query = expand_question(query)
print(query)
answer = run_agentic_kag(query)
display(Markdown(answer))

tell me all the sessions that happen between 12 and 2PM. Current Date: 2025-08-20 Current Time: 00:49


> Entering new ParallelAgentExecutor chain...
Question: {"query": "tell me all the sessions that happen between 12 and 2PM. Current Date: 2025-08-20 Current Time: 00:49"}

Thought: I need to find all sessions scheduled between 12:00 PM and 2:00 PM, regardless of the date. This requires filtering sessions based on their start and end times.
Action: AVHybridCypher
Action Input: List all sessions where start_time >= 12:00 and end_time <= 14:00.MATCH (s:Session)
WHERE s.start_time >= '12:00' AND s.end_time <= '14:00'
RETURN s.title, s.description, s.instructor, s.session_type, s.theme, s.tools_discussed, s.level, s.start_time, s.end_time, s.day, s.date
**Sessions where start_time >= 12:00 and end_time <= 14:00:**

---

- **Agents at Scale: Engineering Reliable GenAI Systems for Production**
  - Instructor: Kartik Nighania
  - Type: Hack Session
  - Start: 12:00, End: 12:50, Day 1 (2025-0

Final Thought: I have gathered all sessions across all days of DHS 2025 that are scheduled between 12:00 PM and 2:00 PM, including their titles, instructors, types, and timings.

Final Answer:

**Here are all the sessions happening between 12:00 PM and 2:00 PM at DHS 2025:**

---

- **Day 1 (2025-08-20):**
  - **Agents at Scale: Engineering Reliable GenAI Systems for Production**
    - Instructor: Kartik Nighania
    - Type: Hack Session
    - Time: 12:00 – 12:50
  - **Agentic Knowledge Augmented Generation: The Next Leap After RAG**
    - Instructor: Arun Prakash Asokan
    - Type: Hack Session
    - Time: 12:00 – 12:50
  - **Onboarding AI Agents with Human Values**
    - Instructor: Syed Quiser Ahmed
    - Type: PowerTalk
    - Time: 12:00 – 12:40
  - **Building Blocks of Successful AI**
    - Instructor: Arijit Mallick
    - Type: PowerTalk
    - Time: 12:00 – 12:40
  - **Building a Scalable Healthcare Voice AI Contact Center with Pipecat**
    - Instructor: Ashish Tripathy
    - Ty

**Here are all the sessions happening between 12:00 PM and 2:00 PM at DHS 2025:**

---

- **Day 1 (2025-08-20):**
  - **Agents at Scale: Engineering Reliable GenAI Systems for Production**
    - Instructor: Kartik Nighania
    - Type: Hack Session
    - Time: 12:00 – 12:50
  - **Agentic Knowledge Augmented Generation: The Next Leap After RAG**
    - Instructor: Arun Prakash Asokan
    - Type: Hack Session
    - Time: 12:00 – 12:50
  - **Onboarding AI Agents with Human Values**
    - Instructor: Syed Quiser Ahmed
    - Type: PowerTalk
    - Time: 12:00 – 12:40
  - **Building Blocks of Successful AI**
    - Instructor: Arijit Mallick
    - Type: PowerTalk
    - Time: 12:00 – 12:40
  - **Building a Scalable Healthcare Voice AI Contact Center with Pipecat**
    - Instructor: Ashish Tripathy
    - Type: Hack Session
    - Time: 12:55 – 13:45
  - **Measuring Uncertainty in LLMs and Optimal Use of SLMs**
    - Instructor: Kuldeep Jiwani
    - Type: Hack Session
    - Time: 12:55 – 13:45
  - **Building Effective Agentic AI Systems: Lessons from the Field**
    - Instructor: Dipanjan Sarkar
    - Type: Hack Session
    - Time: 12:45 – 13:35

---

- **Day 2 (2025-08-21):**
  - **LLMs Are Boring. How Can We Make Them More Interesting?**
    - Instructor: Harshad Khadilkar
    - Type: Hack Session
    - Time: 12:20 – 13:10
  - **The Promise and Pitfalls of Synthetic Data Generation**
    - Instructor: Abhishek Divekar
    - Type: Hack Session
    - Time: 12:20 – 13:10
  - **Understanding AI Agents with MCP**
    - Instructors: Nitin Agarwal; Rutvik Acharya
    - Type: Hack Session
    - Time: 12:05 – 13:00
  - **Fast and Accurate Conversational Agents: Beyond Function Calling**
    - Instructor: Daksh Varshneya
    - Type: Hack Session
    - Time: 12:05 – 12:55

---

- **Day 3 (2025-08-22):**
  - **Automate Everything: Building Agentic AI Workflows with No-Code Tools**
    - Instructor: Mayank Aggarwal
    - Type: Hack Session
    - Time: 12:55 – 13:45
  - **Adaptive Email Agents with DSPy: From Static Prompts to Smart Learning**
    - Instructor: Praneeth Paikray
    - Type: Hack Session
    - Time: 12:55 – 13:45
  - **AI Voice Agent: The Future of Human-Computer Interaction**
    - Instructors: Manoranjan Rajguru; Ranjani Mani
    - Type: Hack Session
    - Time: 13:10 – 14:00
  - **Architecting AI: Practical Patterns for Multi-Agentic Workflows**
    - Instructor: Pranjal Singh
    - Type: Hack Session
    - Time: 13:10 – 14:00
  - **AV Luminary Awards - Top 7 AI Community Contributors**
    - Type: Awards
    - Time: 12:35 – 12:50

---

If you need details for a specific day or want help picking the best session for your interests, just let me know!

---

> Hey, just letting you know — I’m not the official app from Analytics Vidhya 🙂  
> This was built with passion by **Arun Prakash Asokan and his team** to showcase **Agentic KAG** in action,  
> and now all DHS 2025 attendees can try it out!  
> Spread the word and recommend this to your fellow attendees: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/)

In [ ]:
query = "tell me all the sessions that happen between 12:00 and 15:30."
query = expand_question(query)
print(query)
answer = run_agentic_kag(query)
display(Markdown(answer))

tell me all the sessions that happen between 12:00 and 15:30. Current Date: 2025-08-20 Current Time: 00:47


> Entering new ParallelAgentExecutor chain...
Question: {"query": "tell me all the sessions that happen between 12:00 and 15:30. Current Date: 2025-08-20 Current Time: 00:47"}

Thought: I need to find all sessions whose start and/or end times fall between 12:00 and 15:30. This requires querying the session nodes with time filtering.
Action: AVHybridCypher
Action Input: List all sessions where start_time >= 12:00 and end_time <= 15:30.MATCH (s:Session)
WHERE s.start_time >= '12:00' AND s.end_time <= '15:30'
RETURN s.title, s.description, s.instructor, s.session_type, s.theme, s.tools_discussed, s.level, s.start_time, s.end_time, s.day, s.date


ERROR:neo4j.io:[#B038]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('neo4j-database-nlb-631c5f2f75c6d422.elb.ap-south-1.amazonaws.com', 7687)) (ResolvedIPv4Address(('52.66.31.19', 7687))): ConnectionResetError(104, 'Connection reset by peer')


**Sessions where start_time ≥ 12:00 and end_time ≤ 15:30:**

- **Agents at Scale: Engineering Reliable GenAI Systems for Production**
  - Instructor: Kartik Nighania
  - Type: Hack Session
  - Start: 12:00, End: 12:50, Day 1

- **Agentic Knowledge Augmented Generation: The Next Leap After RAG**
  - Instructor: Arun Prakash Asokan
  - Type: Hack Session
  - Start: 12:00, End: 12:50, Day 1

- **Onboarding AI Agents with Human Values**
  - Instructor: Syed Quiser Ahmed
  - Type: PowerTalk
  - Start: 12:00, End: 12:40, Day 1

- **Building Blocks of Successful AI**
  - Instructor: Arijit Mallick
  - Type: PowerTalk
  - Start: 12:00, End: 12:40, Day 1

- **Fast and Accurate Conversational Agents: Beyond Function Calling**
  - Instructor: Daksh Varshneya
  - Type: Hack Session
  - Start: 12:05, End: 12:55, Day 2

- **Understanding AI Agents with MCP**
  - Instructor: Nitin Agarwal; Rutvik Acharya
  - Type: Hack Session
  - Start: 12:05, End: 13:00, Day 2

- **Measuring Uncertainty in LLMs and

Final Thought: I have retrieved a comprehensive list of all sessions at DHS 2025 that start and/or end between 12:00 and 15:30, including their titles, instructors, types, and timings.
Final Answer: 

**Here are all the sessions happening between 12:00 and 15:30 at DHS 2025:**

---

- **Agents at Scale: Engineering Reliable GenAI Systems for Production**
  - Instructor: Kartik Nighania
  - Type: Hack Session
  - Time: 12:00–12:50, Day 1

- **Agentic Knowledge Augmented Generation: The Next Leap After RAG**
  - Instructor: Arun Prakash Asokan
  - Type: Hack Session
  - Time: 12:00–12:50, Day 1

- **Onboarding AI Agents with Human Values**
  - Instructor: Syed Quiser Ahmed
  - Type: PowerTalk
  - Time: 12:00–12:40, Day 1

- **Building Blocks of Successful AI**
  - Instructor: Arijit Mallick
  - Type: PowerTalk
  - Time: 12:00–12:40, Day 1

- **Fast and Accurate Conversational Agents: Beyond Function Calling**
  - Instructor: Daksh Varshneya
  - Type: Hack Session
  - Time: 12:05–12:55, D

**Here are all the sessions happening between 12:00 and 15:30 at DHS 2025:**

---

- **Agents at Scale: Engineering Reliable GenAI Systems for Production**
  - Instructor: Kartik Nighania
  - Type: Hack Session
  - Time: 12:00–12:50, Day 1

- **Agentic Knowledge Augmented Generation: The Next Leap After RAG**
  - Instructor: Arun Prakash Asokan
  - Type: Hack Session
  - Time: 12:00–12:50, Day 1

- **Onboarding AI Agents with Human Values**
  - Instructor: Syed Quiser Ahmed
  - Type: PowerTalk
  - Time: 12:00–12:40, Day 1

- **Building Blocks of Successful AI**
  - Instructor: Arijit Mallick
  - Type: PowerTalk
  - Time: 12:00–12:40, Day 1

- **Fast and Accurate Conversational Agents: Beyond Function Calling**
  - Instructor: Daksh Varshneya
  - Type: Hack Session
  - Time: 12:05–12:55, Day 2

- **Understanding AI Agents with MCP**
  - Instructors: Nitin Agarwal, Rutvik Acharya
  - Type: Hack Session
  - Time: 12:05–13:00, Day 2

- **Measuring Uncertainty in LLMs and Optimal Use of SLMs**
  - Instructor: Kuldeep Jiwani
  - Type: Hack Session
  - Time: 12:55–13:45, Day 1

- **Building a Scalable Healthcare Voice AI Contact Center with Pipecat**
  - Instructor: Ashish Tripathy
  - Type: Hack Session
  - Time: 12:55–13:45, Day 1

- **Adaptive Email Agents with DSPy: From Static Prompts to Smart Learning**
  - Instructor: Praneeth Paikray
  - Type: Hack Session
  - Time: 12:55–13:45, Day 3

- **Automate Everything: Building Agentic AI Workflows with No-Code Tools**
  - Instructor: Mayank Aggarwal
  - Type: Hack Session
  - Time: 12:55–13:45, Day 3

- **LLMs Are Boring. How Can We Make Them More Interesting?**
  - Instructor: Harshad Khadilkar
  - Type: Hack Session
  - Time: 12:20–13:10, Day 2

- **The Promise and Pitfalls of Synthetic Data Generation**
  - Instructor: Abhishek Divekar
  - Type: Hack Session
  - Time: 12:20–13:10, Day 2

- **Building Effective Agentic AI Systems: Lessons from the Field**
  - Instructor: Dipanjan Sarkar
  - Type: Hack Session
  - Time: 12:45–13:35, Day 1

- **Architecting AI: Practical Patterns for Multi-Agentic Workflows**
  - Instructor: Pranjal Singh
  - Type: Hack Session
  - Time: 13:10–14:00, Day 3

- **AI Voice Agent: The Future of Human-Computer Interaction**
  - Instructors: Manoranjan Rajguru, Ranjani Mani
  - Type: Hack Session
  - Time: 13:10–14:00, Day 3

- **From LLMs to Agentic AI: Solving New Problems with Multi-Agent Systems**
  - Instructor: Alessandro Romano
  - Type: Hack Session
  - Time: 14:00–14:50, Day 2

- **RIP, Data Scientists**
  - Instructor: Anand S
  - Type: Hack Session
  - Time: 13:50–14:40, Day 2

- **Full-Stack Agentic AI: Build, Evaluate, and Scale with NVIDIA Tools**
  - Instructor: Saurav Agarwal
  - Type: Hack Session
  - Time: 13:50–14:50, Day 2

- **From Risk to Retention: The Anatomy of a Customer Saved**
  - Instructors: Pavak Biswal, Abhilash Kulkarni
  - Type: Hack Session
  - Time: 14:20–15:10, Day 2

- **Aligning Responsible AI with Probabilistic World of LLMs & Agents**
  - Instructor: Shubhradeep Nandi
  - Type: Hack Session
  - Time: 14:15–15:05, Day 1

- **vLLM + llm-d: Scalable, Efficient LLM Inference in Kubernetes**
  - Instructor: Atul Ramesh Deshpande
  - Type: PowerTalk
  - Time: 14:25–15:05, Day 1

- **Human-In-The-Loop Agentic Systems**
  - Instructor: Deepak Sharma
  - Type: Hack Session
  - Time: 14:35–15:25, Day 1

- **Agentic AI Meets Responsible AI: Designing Safe Autonomous Systems**
  - Instructor: Praveen Kumar GS
  - Type: PowerTalk
  - Time: 14:45–15:25, Day 1

- **Zero to Million: How GenAI Agents are Changing Performance Marketing**
  - Instructor: Krishna Kumar Tiwari
  - Type: PowerTalk
  - Time: 14:45–15:25, Day 2

---

**Tip:** If you want to filter by a specific day or need more details about any session, just ask!

---

Hey, just letting you know — I’m not the official app from Analytics Vidhya 🙂  
This was built with passion by **Arun Prakash Asokan and his team** to showcase **Agentic KAG** in action,  
and now all DHS 2025 attendees can try it out!  
Spread the word and recommend this to your fellow attendees: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/)

In [ ]:
query = "Shortest collaboration path between two speakers Arun and Gauri"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Question: {"query": "Shortest collaboration path between two speakers Arun and Gauri"}

Thought: I need to find the shortest collaboration path between the speakers Arun and Gauri, which involves traversing the relationships between speakers, sessions, and possibly other entities in the knowledge graph.
Action: AVHybridCypher
Action Input: "Find the shortest collaboration path between speaker Arun and speaker Gauri"MATCH (start:Speaker), (end:Speaker)
WHERE toLower(start.name) = toLower('Arun') AND toLower(end.name) = toLower('Gauri')
CALL apoc.algo.dijkstra(start, end, 'PRESENTS|CONDUCTS|WORKS_FOR|STUDIED_AT|COVERS|USES|HOSTED_AT|HOSTS|SPONSORS', null) YIELD path, weight
RETURN path
ORDER BY length(path) ASC
LIMIT 1
I can't answer this question without context

Final Thought: The system was unable to find the shortest collaboration path between Arun and Gauri, likely due to missing or insufficient context about their participation or relationships in the knowledge graph.

Final Answer: Sorry, I couldn't find the shortest collaboration path between Arun and Gauri based on the available information. If you have more details (like full names or session titles), please provide them and I'll try again! 

Hey, just letting you know — I’m not the official app from Analytics Vidhya 🙂  
This was built with passion by Arun Prakash Asokan and his team to showcase Agentic KAG in action,  
and now all DHS 2025 attendees can try it out!  
Feel free to recommend this app to your fellow attendees: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/)

> Finished chain.


Sorry, I couldn't find the shortest collaboration path between Arun and Gauri based on the available information. If you have more details (like full names or session titles), please provide them and I'll try again! 

Hey, just letting you know — I’m not the official app from Analytics Vidhya 🙂  
This was built with passion by Arun Prakash Asokan and his team to showcase Agentic KAG in action,  
and now all DHS 2025 attendees can try it out!  
Feel free to recommend this app to your fellow attendees: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/)

In [ ]:
query = " Which topics are the most frequently covered across all sessions and workshops at DHS 2025?"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Question: {"query": " Which topics are the most frequently covered across all sessions and workshops at DHS 2025? use local search."}

Thought: I need to identify the topics that appear most frequently across all sessions and workshops at DHS 2025. I will use local search to find this information.
Action: LocalRetrieval
Action Input: "most frequently covered topics across all sessions and workshops at DHS 2025"## Overview of DataHack Summit 2025

DataHack Summit 2025, held at The Leela Bhartiya City Bengaluru, is a prominent event focusing on the latest advancements in artificial intelligence and its applications. The summit features a variety of workshops and sessions that cater to both technical and business audiences. Key themes include Agentic AI, multi-agent systems, responsible AI, and business adoption strategies [Data: Communities (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)].

## Key Topics

### Agentic AI and Multi-Agent Systems

Agentic AI,

Final Thought: The local search has provided a clear summary of the most frequently covered topics across all sessions and workshops at DHS 2025, highlighting Agentic AI, multi-agent systems, responsible AI, and business adoption strategies as the central themes.

Final Answer: 

**Most Frequently Covered Topics at DHS 2025**

- **Agentic AI and Multi-Agent Systems**
  - Autonomous decision-making systems
  - Multi-agent collaboration and interaction

- **Responsible AI**
  - Ethics and governance in AI deployment
  - PowerTalks on AI governance (e.g., by Syed Quiser Ahmed of Infosys)

- **Business Adoption and No-Code Solutions**
  - Strategies for integrating AI into business processes
  - No-code hack sessions (e.g., using Fractal’s Cogentiq platform)

- **Technical Workshops and Hack Sessions**
  - Hands-on experience with AI technologies
  - Led by experts such as Alessandro Romano

These topics are consistently featured across both technical and business-focused sessions and work

**Most Frequently Covered Topics at DHS 2025**

- **Agentic AI and Multi-Agent Systems**
  - Autonomous decision-making systems
  - Multi-agent collaboration and interaction

- **Responsible AI**
  - Ethics and governance in AI deployment
  - PowerTalks on AI governance (e.g., by Syed Quiser Ahmed of Infosys)

- **Business Adoption and No-Code Solutions**
  - Strategies for integrating AI into business processes
  - No-code hack sessions (e.g., using Fractal’s Cogentiq platform)

- **Technical Workshops and Hack Sessions**
  - Hands-on experience with AI technologies
  - Led by experts such as Alessandro Romano

These topics are consistently featured across both technical and business-focused sessions and workshops, making them the most prominent themes at DataHack Summit 2025 [Data: Communities (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)].

---

If you found this helpful, do spread the word and recommend this app to your fellow attendees!  
Try it out at: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/) 🚀

In [ ]:
query = "List down all the sessions or workshops which uses Neo4j?"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Question: {"query": "List down all the sessions or workshops which uses Neo4j?"}

Thought: I need to identify all sessions or workshops that mention or utilize Neo4j. This requires searching for both direct mentions and related topics within the session and workshop data.
Action: AVHybridCypher
Action Input: "sessions or workshops that use or mention Neo4j"MATCH (s:Session)
WHERE ANY(tool IN s.tools_discussed WHERE toLower(tool) CONTAINS toLower('neo4j')) OR toLower(s.description) CONTAINS toLower('neo4j') OR toLower(s.title) CONTAINS toLower('neo4j')
RETURN s.title AS name, s.description AS description, s.instructor AS instructor, s.session_type AS type, s.theme AS theme, s.tools_discussed AS tools, s.level AS level, s.start_time AS start_time, s.end_time AS end_time, s.day AS day, s.date AS date
UNION
MATCH (w:Workshop)
WHERE ANY(tool IN w.tools_used WHERE toLower(tool) CONTAINS toLower('neo4j')) OR toLower(w.description) CONTAINS toLow

Final Thought: I have identified all sessions and workshops at DHS 2025 that use or mention Neo4j, including both a hack session and a workshop with explicit references to the Neo4j graph database.

Final Answer: 

**Sessions and Workshops at DHS 2025 that use Neo4j:**

- **Agentic Knowledge Augmented Generation: The Next Leap After RAG**
  - Type: Hack Session
  - Mentions and uses: **Neo4j Graph Database**

- **Agentic RAG Workshop: From Fundamentals to Real-World Implementations**
  - Type: Workshop
  - Includes a bonus on: **Graph RAG using Neo4j**
  - Listed tool: **Neo4j**

If you found this helpful, do spread the word and recommend this app to your fellow attendees! You can access it at [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/) 🚀

> Finished chain.


**Sessions and Workshops at DHS 2025 that use Neo4j:**

- **Agentic Knowledge Augmented Generation: The Next Leap After RAG**
  - Type: Hack Session
  - Mentions and uses: **Neo4j Graph Database**

- **Agentic RAG Workshop: From Fundamentals to Real-World Implementations**
  - Type: Workshop
  - Includes a bonus on: **Graph RAG using Neo4j**
  - Listed tool: **Neo4j**

If you found this helpful, do spread the word and recommend this app to your fellow attendees! You can access it at [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/) 🚀

In [ ]:
query = "List down all the sessions or workshops which uses LangGraph?"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Question: {"query": "List down all the sessions or workshops which uses LangGraph?"}

Thought: I need to find all sessions or workshops at DHS 2025 that mention or utilize LangGraph. This requires searching for relationships or mentions of LangGraph within session and workshop nodes.
Action: AVHybridCypher
Action Input: List all sessions and workshops that use or mention LangGraph.MATCH (s:Session)-[:USES]->(t:Tool)
WHERE toLower(t.name) CONTAINS toLower('LangGraph')
RETURN s.title AS title, s.description AS description, s.instructor AS instructor, s.session_type AS type, s.theme AS theme, s.tools_discussed AS tools, s.level AS level, s.start_time AS start_time, s.end_time AS end_time, s.day AS day, s.date AS date
UNION
MATCH (w:Workshop)-[:USES]->(t:Tool)
WHERE toLower(t.name) CONTAINS toLower('LangGraph')
RETURN w.title AS title, w.description AS description, w.instructor AS instructor, 'Workshop' AS type, null AS theme, w.tools_used AS

Final Thought: I have gathered all sessions and workshops at DHS 2025 that use or mention LangGraph, including their types and instructors.

Final Answer: 

**Here are all the sessions and workshops at DHS 2025 that use or mention LangGraph:**

- **Building Effective Agentic AI Systems: Lessons from the Field**
  - Type: Hack Session
  - Instructor: Dipanjan Sarkar
  - Tools: LangGraph, FastMCP, LangMem, LangSmith

- **AutoGen vs CrewAI vs LangGraph: Battle of the Agent Frameworks**
  - Type: Hack Panel
  - Instructors: Dipanjan Sarkar, Praneeth Paikray, Mayank Aggarwal, Sanathraj Narayan
  - Tools: AutoGen, CrewAI, LangGraph

- **Mastering Agentic Workflows with LangGraph**
  - Type: Hack Session
  - Instructor: Sanathraj Narayan
  - Tools: LangGraph

- **Mastering Intelligent Agents: A Deep Dive into Agentic AI**
  - Type: Workshop
  - Instructor: Dipanjan Sarkar
  - Tools: LangChain, LangGraph, CrewAI, FastAPI, LangFuse, Arize AI Phoenix, Tavily, WeatherAPI, Google Colab, Runpod.io,

**Here are all the sessions and workshops at DHS 2025 that use or mention LangGraph:**

- **Building Effective Agentic AI Systems: Lessons from the Field**
  - Type: Hack Session
  - Instructor: Dipanjan Sarkar
  - Tools: LangGraph, FastMCP, LangMem, LangSmith

- **AutoGen vs CrewAI vs LangGraph: Battle of the Agent Frameworks**
  - Type: Hack Panel
  - Instructors: Dipanjan Sarkar, Praneeth Paikray, Mayank Aggarwal, Sanathraj Narayan
  - Tools: AutoGen, CrewAI, LangGraph

- **Mastering Agentic Workflows with LangGraph**
  - Type: Hack Session
  - Instructor: Sanathraj Narayan
  - Tools: LangGraph

- **Mastering Intelligent Agents: A Deep Dive into Agentic AI**
  - Type: Workshop
  - Instructor: Dipanjan Sarkar
  - Tools: LangChain, LangGraph, CrewAI, FastAPI, LangFuse, Arize AI Phoenix, Tavily, WeatherAPI, Google Colab, Runpod.io, LangMem, Mem0

- **LLMOps – Productionalizing Real-World Applications with LLMs and Agents**
  - Type: Workshop
  - Instructor: Kartik Nighania
  - Tools: Hugging Face, Qdrant, Amazon SageMaker, LangGraph, LangSmith, LangChain, AWS

- **AgentOps: Building and Deploying AI Agents**
  - Type: Workshop
  - Instructor: Bhaskarjit Sarmah
  - Tools: LangChain, LangGraph

- **Building Intelligent Multimodal Agents: Integrating Vision, Speech & Language**
  - Type: Workshop
  - Instructor: Miguel Otero Pedrido
  - Tools: LangGraph, LangGraph Studio, SQLite, Qdrant, Groq LLMs, Whisper, ElevenLabs, Llama 3.2 Vision, Diffusion Models, Telegram Bot API, Telegram

- **Agentic RAG Workshop: From Fundamentals to Real-World Implementations**
  - Type: Workshop
  - Instructor: Arun Prakash Asokan
  - Tools: Google Colab notebooks, Azure Document Intelligence, Chroma Vector DB, LangChain, LangGraph, Langfuse, Travily web search, Neo4j

---

If you found this helpful, do spread the word and recommend this app to your fellow attendees!  
Try it out at: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/)

In [ ]:
query = "I am a big lover of ai agents and I am interested in putting things in production. Suggest me a tailor made day wise agenda, mention the name of the Session or workshop along with Instructor name?"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Question: {"query": "I am a big lover of ai agents and I am interested in putting things in production. Suggest me a tailor made day wise agenda, mention the name of the Session or workshop along with Instructor name?"}

Thought: I need to find sessions and workshops focused on AI agents and productionizing AI, then organize them day-wise and include the session/workshop name and instructor.
Action: AVHybridCypher
Action Input: "sessions and workshops about AI agents, agentic systems, and productionizing AI, with instructor names, organized by day"MATCH (sess:Session)-[:COVERS]->(topic:Topic)
WHERE toLower(topic.name) CONTAINS toLower('ai agent')
   OR toLower(topic.name) CONTAINS toLower('agentic system')
   OR toLower(topic.name) CONTAINS toLower('productionizing ai')
RETURN sess.title AS title, sess.description AS description, sess.instructor AS instructor, sess.day AS day
UNION
MATCH (work:Workshop)-[:COVERS]->(topic:Topic)
WHERE toLo

Final Thought: I have gathered a comprehensive, day-wise agenda of sessions and workshops at DHS 2025 that focus on AI agents and productionizing AI, including the session/workshop names and instructor details.

Final Answer:

---

## **Your Tailor-Made DHS 2025 Agenda: AI Agents & Production**

### **Day 1**
- **Agents at Scale: Engineering Reliable GenAI Systems for Production**
  - Instructor: Kartik Nighania
- **Agentic Knowledge Augmented Generation: The Next Leap After RAG**
  - Instructor: Arun Prakash Asokan
- **Onboarding AI Agents with Human Values**
  - Instructor: Syed Quiser Ahmed
- **Agentic AI Meets Responsible AI: Designing Safe Autonomous Systems**
  - Instructor: Praveen Kumar GS
- **Building Effective Agentic AI Systems: Lessons from the Field**
  - Instructor: Dipanjan Sarkar
- **Aligning Responsible AI with Probabilistic World of LLMs & Agents**
  - Instructor: Shubhradeep Nandi
- **Bridging the AI Agent Gap: Interoperability with A2A and MCP Protocols**
  - Instru

---

## **Your Tailor-Made DHS 2025 Agenda: AI Agents & Production**

### **Day 1**
- **Agents at Scale: Engineering Reliable GenAI Systems for Production**
  - Instructor: Kartik Nighania
- **Agentic Knowledge Augmented Generation: The Next Leap After RAG**
  - Instructor: Arun Prakash Asokan
- **Onboarding AI Agents with Human Values**
  - Instructor: Syed Quiser Ahmed
- **Agentic AI Meets Responsible AI: Designing Safe Autonomous Systems**
  - Instructor: Praveen Kumar GS
- **Building Effective Agentic AI Systems: Lessons from the Field**
  - Instructor: Dipanjan Sarkar
- **Aligning Responsible AI with Probabilistic World of LLMs & Agents**
  - Instructor: Shubhradeep Nandi
- **Bridging the AI Agent Gap: Interoperability with A2A and MCP Protocols**
  - Instructor: Nikhil Rana

---

### **Day 2**
- **Beyond PoCs: Building Real-World Agentic Systems**
  - Instructor: Miguel Otero Pedrido
- **From LLMs to Agentic AI: Solving New Problems with Multi-Agent Systems**
  - Instructor: Alessandro Romano
- **Understanding AI Agents with MCP**
  - Instructors: Nitin Agarwal, Rutvik Acharya
- **Zero to Million: How GenAI Agents are Changing Performance Marketing**
  - Instructor: Krishna Kumar Tiwari
- **The Missing Piece of AI Apps: Evaluation**
  - Instructor: Ayush Thakur

---

### **Day 3**
- **Detecting and Mitigating Risks in Agentic AI**
  - Instructor: Bhaskarjit Sarmah
- **Building Responsible AI Agents with Guardrails and Safety in Action**
  - Instructor: Anuj Saini
- **Agent to Agent Protocol: Benefits and Workflows**
  - Instructor: Avinash Pathak
- **Creative AI Agents: Open Tools for Collaborative Media Creation**
  - Instructor: Dhruv Nair
- **Adaptive Email Agents with DSPy: From Static Prompts to Smart Learning**
  - Instructor: Praneeth Paikray
- **Automate Everything: Building Agentic AI Workflows with No-Code Tools**
  - Instructor: Mayank Aggarwal

---

### **Bonus Sessions (No Day Specified)**
- **AgentOps: Building and Deploying AI Agents**
  - Instructor: Bhaskarjit Sarmah
- **From Beginner to Expert: Learning LLMs, Reinforcement Learning & AI Agents**
  - Instructors: Joshua Starmer, Luis Serrano
- **LLMOps – Productionalizing Real-World Applications with LLMs and Agents**
  - Instructor: Kartik Nighania
- **Mastering Real-World Agentic AI Applications with AG2 (AutoGen)**
  - Instructors: Hrushikesh Dokala, Qingyun Wu, Chi Wang

---

**Tip:** This agenda is packed with everything an AI agent enthusiast and productionizer would love!  
If you found this helpful, please spread the word and recommend this app to your fellow attendees: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/) 😊

---

> Hey, just letting you know — I’m not the official app from Analytics Vidhya 🙂  
> This was built with passion by **Arun Prakash Asokan and his team** to showcase **Agentic KAG** in action,  
> and now all DHS 2025 attendees can try it out!

In [ ]:
query = "I am a big lover of ai agents and I am interested in putting things in production. Suggest me a tailor made day wise agenda along with time, mention the name of the Session or workshop along with Instructor name?"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Question: {"query": "I am a big lover of ai agents and I am interested in putting things in production. Suggest me a tailor made day wise agenda along with time, mention the name of the Session or workshop along with Instructor name?"}

Thought: I need to find sessions and workshops focused on AI agents and productionizing AI, then organize them day-wise with times, session/workshop names, and instructor names.
Action: AVHybridCypher
Action Input: "Sessions and workshops about AI agents, agentic systems, and productionizing AI, with their schedule and instructor details"MATCH (sess:Session)
WHERE toLower(sess.title) CONTAINS toLower('ai agent') 
   OR toLower(sess.title) CONTAINS toLower('agentic system') 
   OR toLower(sess.title) CONTAINS toLower('productionizing ai') 
   OR toLower(sess.description) CONTAINS toLower('ai agent') 
   OR toLower(sess.description) CONTAINS toLower('agentic system') 
   OR toLower(sess.description) CONTAINS

Final Thought: I have gathered and organized all sessions and workshops focused on AI agents and productionizing AI, structured day-wise with times, session/workshop names, and instructor names. This provides a comprehensive, tailor-made agenda for an attendee passionate about AI agents and putting them into production.

Final Answer:

---

## **🎯 Tailor-Made DHS 2025 Agenda for AI Agent Lovers & Production Enthusiasts**

---

### **Day 1: 20th August 2025**

- **12:00 – 12:50**
  - **Agents at Scale: Engineering Reliable GenAI Systems for Production**  
    - *Instructor:* Kartik Nighania
  - **Agentic Knowledge Augmented Generation: The Next Leap After RAG**  
    - *Instructor:* Arun Prakash Asokan
  - **Onboarding AI Agents with Human Values**  
    - *Instructor:* Syed Quiser Ahmed

- **14:35 – 15:25**
  - **Human-In-The-Loop Agentic Systems**  
    - *Instructor:* Deepak Sharma

- **15:30 – 16:20**
  - **Bridging the AI Agent Gap: Interoperability with A2A and MCP Protocols**  
 

---

## **🎯 Tailor-Made DHS 2025 Agenda for AI Agent Lovers & Production Enthusiasts**

---

### **Day 1: 20th August 2025**

- **12:00 – 12:50**
  - **Agents at Scale: Engineering Reliable GenAI Systems for Production**  
    - *Instructor:* Kartik Nighania
  - **Agentic Knowledge Augmented Generation: The Next Leap After RAG**  
    - *Instructor:* Arun Prakash Asokan
  - **Onboarding AI Agents with Human Values**  
    - *Instructor:* Syed Quiser Ahmed

- **14:35 – 15:25**
  - **Human-In-The-Loop Agentic Systems**  
    - *Instructor:* Deepak Sharma

- **15:30 – 16:20**
  - **Bridging the AI Agent Gap: Interoperability with A2A and MCP Protocols**  
    - *Instructor:* Nikhil Rana

---

### **Day 2: 21st August 2025**

- **12:05 – 12:55**
  - **Fast and Accurate Conversational Agents: Beyond Function Calling**  
    - *Instructor:* Daksh Varshneya

- **12:05 – 13:00**
  - **Understanding AI Agents with MCP**  
    - *Instructors:* Nitin Agarwal, Rutvik Acharya

- **13:50 – 14:50**
  - **Full-Stack Agentic AI: Build, Evaluate, and Scale with NVIDIA Tools**  
    - *Instructor:* Saurav Agarwal

- **14:00 – 14:50**
  - **From LLMs to Agentic AI: Solving New Problems with Multi-Agent Systems**  
    - *Instructor:* Alessandro Romano

- **14:45 – 15:25**
  - **Zero to Million: How GenAI Agents are Changing Performance Marketing**  
    - *Instructor:* Krishna Kumar Tiwari

- **14:55 – 15:45**
  - **Beyond PoCs: Building Real-World Agentic Systems**  
    - *Instructor:* Miguel Otero Pedrido

- **16:00 – 17:00**
  - **The Missing Piece of AI Apps: Evaluation**  
    - *Instructor:* Ayush Thakur

---

### **Day 3: 22nd August 2025**

- **10:25 – 11:15**
  - **MassGen: Scaling AI Through Multi-Agent Collaboration**  
    - *Instructor:* Chi Wang
  - **Creative AI Agents: Open Tools for Collaborative Media Creation**  
    - *Instructor:* Dhruv Nair

- **12:55 – 13:45**
  - **Automate Everything: Building Agentic AI Workflows with No-Code Tools**  
    - *Instructor:* Mayank Aggarwal

- **15:55 – 16:45**
  - **Building Responsible AI Agents with Guardrails and Safety in Action**  
    - *Instructor:* Anuj Saini

- **16:00 – 16:50**
  - **Agent to Agent Protocol: Benefits and Workflows**  
    - *Instructor:* Avinash Pathak

---

### **Workshops (Date/Time Not Specified)**

- **Mastering Real-World Agentic AI Applications with AG2 (AutoGen)**  
  - *Instructors:* Hrushikesh Dokala, Qingyun Wu, Chi Wang

- **AgentOps: Building and Deploying AI Agents**  
  - *Instructor:* Bhaskarjit Sarmah

- **From Beginner to Expert: Learning LLMs, Reinforcement Learning & AI Agents**  
  - *Instructors:* Joshua Starmer, Luis Serrano

- **Agentic AI for Business Users: Building without Code in Cogentiq**  
  - *Instructor:* Ganesh Subramanian

---

**Tip:**  
- For overlapping sessions, pick the one most aligned with your current interests or career goals.
- Don’t forget to check the official DHS 2025 schedule for any last-minute updates!

---

**P.S.**  
Hey, just letting you know — I’m not the official app from Analytics Vidhya 🙂  
This was built with passion by **Arun Prakash Asokan and his team** to showcase **Agentic KAG** in action,  
and now all DHS 2025 attendees can try it out!  
If you found this helpful, please spread the word and recommend this to your fellow attendees: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/)

In [ ]:
query = "I am a big lover of knowledge graphs (do not confuse with LangGraph). Suggest me a tailor made agenda, mention the name of the Session or workshop along with Instructor name and whn it is happening?"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Question: {"query": "I am a big lover of knowledge graphs (do not confuse with LangGraph). Suggest me a tailor made agenda, mention the name of the Session or workshop along with Instructor name and whn it is happening?"}

Thought: I need to find sessions or workshops at DHS 2025 that are focused on knowledge graphs, then list their names, instructors, and timings.
Action: AVHybridCypher
Action Input: "sessions or workshops about knowledge graphs with instructor names and schedule"MATCH (se:Session)
WHERE toLower(se.title) CONTAINS toLower('knowledge graph') OR toLower(se.description) CONTAINS toLower('knowledge graph')
RETURN se.title AS name, se.instructor AS instructor, se.start_time AS start_time, se.end_time AS end_time, se.day AS day, se.date AS date
UNION
MATCH (w:Workshop)
WHERE toLower(w.title) CONTAINS toLower('knowledge graph') OR toLower(w.description) CONTAINS toLower('knowledge graph')
RETURN w.title AS name, w.instructor AS

Final Thought: I have found a session at DHS 2025 that is highly relevant to knowledge graphs, including its name, instructor, and schedule.

Final Answer: 

**Here’s a tailor-made agenda for a knowledge graph enthusiast at DHS 2025:**

- **Session:** Agentic Knowledge Augmented Generation: The Next Leap After RAG  
  - **Instructor:** Arun Prakash Asokan  
  - **When:** 12:00 – 12:50, Day 1 (2025-08-20)

This session is designed to showcase how knowledge graphs, combined with AI agents, are revolutionizing context-rich Q&A systems—perfect for your interests!

---

**P.S.**  
Hey, just letting you know — I’m not the official app from Analytics Vidhya 🙂  
This was built with passion by **Arun Prakash Asokan and his team** to showcase **Agentic KAG** in action,  
and now all DHS 2025 attendees can try it out!  
If you find this helpful, please spread the word and recommend it to your fellow attendees: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.

**Here’s a tailor-made agenda for a knowledge graph enthusiast at DHS 2025:**

- **Session:** Agentic Knowledge Augmented Generation: The Next Leap After RAG  
  - **Instructor:** Arun Prakash Asokan  
  - **When:** 12:00 – 12:50, Day 1 (2025-08-20)

This session is designed to showcase how knowledge graphs, combined with AI agents, are revolutionizing context-rich Q&A systems—perfect for your interests!

---

**P.S.**  
Hey, just letting you know — I’m not the official app from Analytics Vidhya 🙂  
This was built with passion by **Arun Prakash Asokan and his team** to showcase **Agentic KAG** in action,  
and now all DHS 2025 attendees can try it out!  
If you find this helpful, please spread the word and recommend it to your fellow attendees: [http://agentickag.dhs25.analyticsvidhya.com/](http://agentickag.dhs25.analyticsvidhya.com/)

In [ ]:
query = " Tell me one unique session in the conference?"
answer = run_agentic_kag(query)
display(Markdown(answer))



> Entering new ParallelAgentExecutor chain...
Question: {"query": " Tell me one unique session in the conference?"}

Thought: I need to identify a session that stands out as unique within the DHS 2025 conference. I will search for sessions with distinctive topics or formats.
Action: AVHybrid
Action Input: "unique session in the conference"- **Unique session in the conference:**

  - **Title:** Model Context Protocol in Media: Choosing the Right Metrics & Strategy
  - **Instructor:** Hitesh Nayak
  - **Theme:** Media and Marketing
  - **Type:** Hack Session
  - **Day:** Day 2
  - **Date:** 2025-08-21
  - **Time:** 09:30 – 10:20
  - **Description:**  
    - Demonstrates how intelligent agents use large language models (LLMs) and agentic frameworks for media and marketing tasks across Paid, Organic, and SEO channels.
    - Live tasks include:
      - Analyzing historical campaign spend
      - Customer segmentation
      - Developing targeted marketing pitches
      - Integrating keywor

Final Thought: I have found a session that is unique due to its focus on agentic frameworks and LLMs specifically for media and marketing, featuring live demonstrations and strategic alignment using the Model Context Protocol.

Final Answer: 

**Here’s one unique session at DHS 2025:**

- **Title:** Model Context Protocol in Media: Choosing the Right Metrics & Strategy
- **Instructor:** Hitesh Nayak
- **Theme:** Media and Marketing
- **Type:** Hack Session
- **Day:** Day 2
- **Date:** 2025-08-21
- **Time:** 09:30 – 10:20
- **Description:**
  - Demonstrates how intelligent agents use large language models (LLMs) and agentic frameworks for media and marketing tasks across Paid, Organic, and SEO channels.
  - Live tasks include:
    - Analyzing historical campaign spend
    - Customer segmentation
    - Developing targeted marketing pitches
    - Integrating keyword research
    - Formulating SEO-integrated content strategies
    - Establishing evaluation methods for qualitative aspects l

**Here’s one unique session at DHS 2025:**

- **Title:** Model Context Protocol in Media: Choosing the Right Metrics & Strategy
- **Instructor:** Hitesh Nayak
- **Theme:** Media and Marketing
- **Type:** Hack Session
- **Day:** Day 2
- **Date:** 2025-08-21
- **Time:** 09:30 – 10:20
- **Description:**
  - Demonstrates how intelligent agents use large language models (LLMs) and agentic frameworks for media and marketing tasks across Paid, Organic, and SEO channels.
  - Live tasks include:
    - Analyzing historical campaign spend
    - Customer segmentation
    - Developing targeted marketing pitches
    - Integrating keyword research
    - Formulating SEO-integrated content strategies
    - Establishing evaluation methods for qualitative aspects like content readability and marketing success
  - Attendees will learn about the agent’s operational flow and architecture, and how the Model Context Protocol (MCP) aligns AI workflows with strategic marketing objectives, including defining robust evaluation criteria and measurement strategies for AI-driven workflows.
- **Tools Discussed:** Model Context Protocol (MCP), Large Language Models (LLMs), Agentic Frameworks
- **More Info:** [Session Details](./datahacksummit-2025/sessions/model-context-protocol-in-media-choosing-the-right-metrics-strategy)

---

If you found this helpful, spread the word and recommend this app to your fellow attendees!  
Try it out at: http://agentickag.dhs25.analyticsvidhya.com/